In [65]:
from sqlalchemy import Column, Integer, String, Date, create_engine, select, Time, Boolean, ForeignKey, UniqueConstraint, CheckConstraint
import sqlalchemy.orm as saorm
from sqlalchemy.exc import IntegrityError
from datetime import datetime, date, timedelta

engine = create_engine("sqlite:///system.db", echo=True)
Base = saorm.declarative_base()
Session = saorm.sessionmaker(bind=engine)
session = Session()

class mitarbeiter(Base):
    __tablename__ = "users"
    mitarbeiter_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(30), unique=True, nullable=False)
    password = Column(String(15), nullable=False)
    vertragliche_wochenstunden = Column(Integer, nullable=False)
    geburtsdatum = Column(Date, nullable=False)
    gleitzeit = Column(Integer, nullable=False, default=0)
    letzter_login = Column(Date, nullable=False)
    ampel_grün = Column(Integer, nullable=False, default=5)
    ampel_rot = Column(Integer, nullable=False, default=-5)

class Abwesenheit(Base):
    __tablename__ = "abwesenheiten"

    id = Column(Integer, primary_key=True, autoincrement=True)
    mitarbeiter_id = Column(Integer, ForeignKey("users.mitarbeiter_id"), nullable=False)
    datum = Column(Date, nullable=False)
    typ = Column(String, CheckConstraint("typ IN ('Urlaub', 'Krankheit', 'Fortbildung', 'Sonstiges')"), nullable=False)
    genehmigt = Column(Boolean, nullable=False, default=False)


class Zeiteintrag(Base):
    __tablename__ = "zeiteinträge"

    id = Column(Integer, primary_key=True, autoincrement=True)
    mitarbeiter_id = Column(Integer, ForeignKey("users.mitarbeiter_id"), nullable=False)
    zeit = Column(Time, nullable=False)
    datum = Column(Date, nullable=False)
    validiert = Column(Boolean, nullable=False, default=False) 

class Benachrichtigungen(Base):
    __tablename__ = "benachrichtigungen"

    id = Column(Integer, primary_key=True, autoincrement=True)
    mitarbeiter_id = Column(Integer, ForeignKey("users.mitarbeiter_id"), nullable=False)
    benachrichtigungs_code = Column(Integer, nullable=False)
    datum = Column(Date)

    CODES = {
        1.1:"An den Tag",
        1.2:"wurde nicht gestempelt. Es wird für jeden Tag die Tägliche arbeitszeit der Gleitzeit abgezogen",
        2.1:"Am",
        2.2:"fehlt ein Stempel, bitte tragen sie diesen nach",
        3: ["Achtung, am", "wurden die gesetzlichen Ruhezeiten nicht eingehalten"],
        4: "Achtung, Ihre durchschnittliche tägliche Arbeitszeit der letzten 6 Monate hat 8 Stunden überschritten."
    }

    __table_args__ = (
        UniqueConstraint("mitarbeiter_id", "benachrichtigungs_code", "datum", name="uq_benachrichtigung_unique"),
    )

    def create_fehlermeldung(self):

        if self.benachrichtigungs_code == 1:
            return f"{self.CODES[1.1]} {self.datum} {self.CODES[1.2]}"
        elif self.benachrichtigungs_code == 2:
            return f"{self.CODES[2.1]} {self.datum} {self.CODES[2.2]}"
        
        elif self.benachrichtigungs_code == 3:
            return f"{self.CODES[3][0]} {self.datum} {self.CODES[3][1]}"
        
        elif self.benachrichtigungs_code ==4:
            return self.CODES[4]



class CalculateTime():
     def __new__(cls, eintrag1, eintrag2):
         if eintrag1.datum != eintrag2.datum:
             return None
         return super().__new__(cls)

     def __init__(self, eintrag1, eintrag2):
         self.datum = eintrag1.datum
         self.startzeit = eintrag1.zeit
         self.endzeit = eintrag2.zeit
         start_dt = datetime.combine(self.datum, self.startzeit)
         end_dt = datetime.combine(self.datum, self.endzeit)

         self.gearbeitete_zeit = end_dt - start_dt  

     def gesetzliche_pausen_hinzufügen(self):
         if self.gearbeitete_zeit > timedelta(hours=6):
             self.gearbeitete_zeit -= timedelta(minutes=30)



class ModellTrackTime():
    def __init__(self):
        self.aktueller_nutzer_id = None
        self.aktueller_nutzer_name = None
        self.aktueller_nutzer_vertragliche_wochenstunden = None
        self.aktueller_nutzer_gleitzeit = None
        self.aktueller_nutzer_ampel_rot = None
        self.aktueller_nutzer_ampel_grün = None

        self.manueller_stempel_datum = None
        self.manueller_stempel_uhrzeit = None

        self.zeiteinträge_bestimmtes_datum = None
        self.bestimmtes_datum = None

        self.neues_passwort = None
        self.neues_passwort_wiederholung = None

        self.ampel_status = None

        self.neuer_abwesenheitseintrag_datum = None
        self.neuer_abwesenheitseintrag_art = None


        self.benachrichtigungen = []


        self.feedback_manueller_stempel = ""
        self.feedback_arbeitstage = ""
        self.feedback_stempel = ""
        self.feedback_neues_passwort = ""

    def get_zeiteinträge(self):
        if self.aktueller_nutzer_id is None or self.bestimmtes_datum is None:
            return
        
        date = datetime.strptime(self.bestimmtes_datum, "%d.%m.%Y").date()


        stmt = select(
            Zeiteintrag
            ).where(
                (Zeiteintrag.mitarbeiter_id == self.aktueller_nutzer_id)&(Zeiteintrag.datum == date)
                ).order_by(
                    Zeiteintrag.datum, Zeiteintrag.zeit
                    )
        einträge = session.scalars(stmt).all()
        self.zeiteinträge_bestimmtes_datum = einträge


    def get_user_info(self):

        if self.aktueller_nutzer_id is None:
            return

        stmt = select(mitarbeiter).where(mitarbeiter.mitarbeiter_id == self.aktueller_nutzer_id)
        nutzer = session.execute(stmt).scalar_one_or_none()
        if nutzer:
            self.aktueller_nutzer_name = nutzer.name
            self.aktueller_nutzer_vertragliche_wochenstunden = nutzer.vertragliche_wochenstunden
            self.aktueller_nutzer_gleitzeit = nutzer.gleitzeit
            self.aktueller_nutzer_ampel_rot = nutzer.ampel_rot
            self.aktueller_nutzer_ampel_grün = nutzer.ampel_grün

    def set_ampel_farbe(self):
        if self.aktueller_nutzer_gleitzeit >= self.aktueller_nutzer_ampel_grün:
            self.ampel_status = "green"

        elif (self.aktueller_nutzer_gleitzeit < self.aktueller_nutzer_ampel_grün) & (self.aktueller_nutzer_gleitzeit > self.aktueller_nutzer_ampel_rot):
            self.ampel_status = "yellow"
    
        else:
            self.ampel_status = "red"


    def get_messages(self):

        stmt = select(Benachrichtigungen).where(Benachrichtigungen.mitarbeiter_id == self.aktueller_nutzer_id)

        result = session.execute(stmt).scalars().all()
        self.benachrichtigungen = result

    def update_passwort(self):
        if not self.neues_passwort:
            self.feedback_neues_passwort = "Bitte gebe ein passwort ein"
            return
        elif  not self.neues_passwort_wiederholung:
            self.feedback_neues_passwort = "Bitte wiederhole das Passwort"
            return
        elif self.neues_passwort != self.neues_passwort_wiederholung:
            self.feedback_neues_passwort = "Die Passwörter müssen übereinstimmen"
            return
        else:
            stmt = select(mitarbeiter).where(mitarbeiter.mitarbeiter_id == self.aktueller_nutzer_id)
            nutzer = session.execute(stmt).scalar_one_or_none()
            nutzer.password = self.neues_passwort
            session.commit()
            self.feedback_neues_passwort = "Passwort erfolgreich geändert"
            return



    def stempel_hinzufügen(self):


        stempel = Zeiteintrag(
            mitarbeiter_id = self.aktueller_nutzer_id,
            zeit = datetime.now().time(),
            datum = date.today()
        )
        session.add(stempel)
        session.commit()

    def manueller_stempel_hinzufügen(self):
        stempel = Zeiteintrag(
            mitarbeiter_id = self.aktueller_nutzer_id,
            zeit =datetime.strptime(self.manueller_stempel_uhrzeit, "%H:%M").time(),
            datum = datetime.strptime(self.manueller_stempel_datum, "%d/%m/%Y").date()
        )
        session.add(stempel)
        session.commit()

        self.feedback_manueller_stempel = f"Stempel am {self.manueller_stempel_datum} um {self.manueller_stempel_uhrzeit} erfolgreich hinzugefügt"


  



   
 


    def berechne_gleitzeit(self):
        stmt = select(Zeiteintrag).where(
            (Zeiteintrag.mitarbeiter_id == self.aktueller_nutzer_id) &
            (Zeiteintrag.validiert == 0) &
            (Zeiteintrag.datum <= date.today() - timedelta(days=1))
        ).order_by(Zeiteintrag.datum, Zeiteintrag.zeit)
        einträge = session.scalars(stmt).all()
        for e in einträge:
            print(e.datum, e.zeit)
 
        arbeitstage ={}
        benutzte_einträge = [] 




        i = 0
        while i < len(einträge) - 1:
            print("test")
            calc = CalculateTime(einträge[i], einträge[i+1])
            if calc:
    
                if calc.datum in arbeitstage:
                    calc.gesetzliche_pausen_hinzufügen()
                    arbeitstage[calc.datum] += calc.gearbeitete_zeit
                else:
                    calc.gesetzliche_pausen_hinzufügen()
                    arbeitstage[calc.datum] = calc.gearbeitete_zeit

                benutzte_einträge.append(einträge[i])
                benutzte_einträge.append(einträge[i+1])
                i += 2  
            else:
                i += 1  
        print("test")

        print(arbeitstage)

        tägliche_arbeitszeit = timedelta( hours=(self.aktueller_nutzer_vertragliche_wochenstunden / 5))

        for datum, arbeitszeit in arbeitstage.items():
            exists_stmt = select(Benachrichtigungen).where(
                (Benachrichtigungen.mitarbeiter_id == self.aktueller_nutzer_id) &
                (Benachrichtigungen.datum == datum) &
                (Benachrichtigungen.benachrichtigungs_code == 1)
            )
            exists = session.execute(exists_stmt).scalars().first()
            print(exists)

            unvalidierte_stmt = select(Zeiteintrag).where(
                (Zeiteintrag.mitarbeiter_id == self.aktueller_nutzer_id) &
                (Zeiteintrag.datum == datum) &
                (Zeiteintrag.validiert.is_(False))
            )
            unvalidierte = session.execute(unvalidierte_stmt).scalars().first()
            print(unvalidierte)

            if (len(exists) == 0) and (len(unvalidierte) == 0):
                print("abzug:", datum)
                arbeitstage[datum] -= tägliche_arbeitszeit


       # for e in benutzte_einträge:
        #    e.validiert = True
         #   session.commit()

        gleitzeit_delta = sum(arbeitstage.values(), timedelta())
        gleitzeit_stunden = float(gleitzeit_delta.total_seconds() / 3600)
        self.aktueller_nutzer_gleitzeit += gleitzeit_stunden


        nutzer = session.get(mitarbeiter, self.aktueller_nutzer_id)
        if nutzer:
            nutzer.gleitzeit = self.aktueller_nutzer_gleitzeit
            session.commit()

   





        


In [2]:
test= {"e": "eins", "z":"zwei"}

for i in test.keys():
    print(i)

e
z


In [66]:
Model = ModellTrackTime()
Model.aktueller_nutzer_id =1
Model.get_user_info()

2025-10-13 19:44:21,004 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-13 19:44:21,006 INFO sqlalchemy.engine.Engine SELECT users.mitarbeiter_id, users.name, users.password, users.vertragliche_wochenstunden, users.geburtsdatum, users.gleitzeit, users.letzter_login, users."ampel_grün", users.ampel_rot 
FROM users 
WHERE users.mitarbeiter_id = ?
2025-10-13 19:44:21,006 INFO sqlalchemy.engine.Engine [generated in 0.00043s] (1,)


In [67]:
Model.berechne_gleitzeit()


2025-10-13 19:44:27,359 INFO sqlalchemy.engine.Engine SELECT "zeiteinträge".id, "zeiteinträge".mitarbeiter_id, "zeiteinträge".zeit, "zeiteinträge".datum, "zeiteinträge".validiert 
FROM "zeiteinträge" 
WHERE "zeiteinträge".mitarbeiter_id = ? AND "zeiteinträge".validiert = ? AND "zeiteinträge".datum <= ? ORDER BY "zeiteinträge".datum, "zeiteinträge".zeit
2025-10-13 19:44:27,360 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (1, 0, '2025-10-12')
2025-09-30 07:00:00
2025-09-30 15:30:00
test
test
{datetime.date(2025, 9, 30): datetime.timedelta(seconds=28800)}
2025-10-13 19:44:27,361 INFO sqlalchemy.engine.Engine SELECT benachrichtigungen.id, benachrichtigungen.mitarbeiter_id, benachrichtigungen.benachrichtigungs_code, benachrichtigungen.datum 
FROM benachrichtigungen 
WHERE benachrichtigungen.mitarbeiter_id = ? AND benachrichtigungen.datum = ? AND benachrichtigungen.benachrichtigungs_code = ?
2025-10-13 19:44:27,362 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1, '2025-09-3

TypeError: object of type 'NoneType' has no len()

In [60]:
Model.aktueller_nutzer_gleitzeit

64.0

In [33]:
Model.aktueller_nutzer_vertragliche_wochenstunden


40

In [28]:
!pytest -v 

============================= test session starts =============================
platform win32 -- Python 3.13.7, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\Sonja\Documents\Fallstudie\fallstudie_venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\Sonja\Documents\Fallstudie
collecting ... collected 29 items

test_modelltracktime_adult.py::test_urlaub_verhindert_gleitzeitabzug PASSED [  3%]
test_modelltracktime_adult.py::test_benachrichtigung_bei_fehlendem_stempel PASSED [  6%]
test_modelltracktime_adult.py::test_keine_doppelten_benachrichtigungen PASSED [ 10%]
test_modelltracktime_adult.py::test_benachrichtigung_fehlt_stempel PASSED [ 13%]
test_modelltracktime_adult.py::test_nachtraeglicher_stempel_an_gestempelten_tag PASSED [ 17%]
test_modelltracktime_adult.py::test_checke_ruhezeiten_verstoss PASSED    [ 20%]
test_modelltracktime_adult.py::test_checke_durchschnittliche_arbeitszeit_zu_lang PASSED [ 24%]
test_modelltracktime_adult.py::test_checke_durchschnittliche_arbeitszeit_ok